In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
# 读入数据
data_batch = "test"
data_target = "price"
path = f"ori_data//ruc_Class25Q2_{data_batch}_{data_target}.csv"
df = pd.read_csv(path)
print(f"数据维度：{df.shape}")

数据维度：(34017, 55)


/tmp/ipykernel_1090/905137227.py:5: DtypeWarning: Columns (4,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [3]:
# 处理“环线”特征：删除该列
df = df.drop("环线", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 54)


In [4]:
# 处理“房屋户型”：“2室1厅1厨1卫”
def extract_room_shape(text):
    """
    参数:
    text: str - 户型字符串，如"2室1厅1厨1卫"
    
    返回:
    tuple - (室数, 厅数, 厨房数, 卫数)
    """
    # 初始化默认值
    if text == "":
        return -1,-1,-1,-1
    room = hall = kitchen = bathroom = 0
    
    # 使用正则表达式匹配数字和对应的单位
    pattern = r'(\d+)(室|厅|厨|卫)'
    matches = re.findall(pattern, text)
    
    # 将匹配结果分配到对应的变量
    for value, unit in matches:
        value = int(value)
        if unit == '室':
            room = value
        elif unit == '厅':
            hall = value
        elif unit == '厨':
            kitchen = value
        elif unit == '卫':
            bathroom = value
    
    return room, hall, kitchen, bathroom

In [5]:
# 检查缺失值
print(df['房屋户型'].isnull().sum())
# 填充缺失值为空字符串
df['房屋户型'] = df['房屋户型'].fillna('')
# 应用提取函数并创建新列
df[['室数', '厅数', '厨房数', '卫数']] = df['房屋户型'].apply(lambda x: pd.Series(extract_room_shape(x)))

print(df.columns)
# print(df.head())
print(f"数据维度：{df.shape}")

14
Index(['ID', '城市', '区域', '板块', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构',
       '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限',
       '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat',
       '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数',
       '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水',
       '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈',
       '室数', '厅数', '厨房数', '卫数'],
      dtype='object')
数据维度：(34017, 58)


In [6]:
df = df.drop("房屋户型", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 57)


In [7]:
# 处理“所在楼层”：“低楼层（共13层）”
def extract_floor_info(text):
    """
    参数:
    text: str - 楼层字符串，如"中楼层（共7层）"、"底层（共13层）"
    
    返回:
    tuple - (是否为底层, 是否为低层, 是否为中层, 是否为高层, 是否为顶层, 总楼层)
    """
    # 初始化默认值
    is_bottom = 0
    is_low = 0
    is_middle = 0
    is_high = 0
    is_top = 0
    total_floors = 0
    
    # 提取总楼层数
    total_match = re.search(r'共(\d+)层', text)
    if total_match:
        total_floors = int(total_match.group(1))
    
    # 根据楼层描述设置对应的标志
    if '底层' in text:
        is_bottom = 1
    elif '低楼层' in text:
        is_low = 1
    elif '中楼层' in text:
        is_middle = 1
    elif '高楼层' in text:
        is_high = 1
    elif '顶层' in text:
        is_top = 1
    
    return is_bottom, is_low, is_middle, is_high, is_top, total_floors

In [8]:
# 检查缺失值
print(df['所在楼层'].isnull().sum())
# 填充缺失值为空字符串
df['所在楼层'] = df['所在楼层'].fillna('')
# 应用提取函数并创建新列
df[['是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层']] = df['所在楼层'].apply(
    lambda x: pd.Series(extract_floor_info(x))
)

# print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
        ID  城市     区域      板块        所在楼层     建筑面积     套内面积 房屋朝向  建筑结构 装修情况  \
0  1000000   0  109.0   367.0  中楼层 (共23层)  282.02㎡      NaN  南 北  钢混结构   简装   
1  1000001   0   28.0   606.0  中楼层 (共17层)   88.42㎡   71.78㎡  南 北  钢混结构   其他   
2  1000002   0  123.0  1110.0  高楼层 (共12层)  175.52㎡  139.86㎡   西北  混合结构   简装   
3  1000003   0   65.0   555.0   中楼层 (共5层)  106.13㎡      NaN  南 北  钢混结构   其他   
4  1000004   0  109.0   990.0    顶层 (共5层)   116.8㎡      NaN  南 北  混合结构   简装   

   ... 室数 厅数 厨房数 卫数 是否为底层 是否为低层 是否为中层 是否为高层 是否为顶层  总楼层  
0  ...  3  2   1  2     0     0     1     0     0   23  
1  ...  2  1   1  1     0     0     1     0     0   17  
2  ...  3  1   1  2     0     0     0     1     0   12  
3  ...  2  1   1  1     0     0     1     0     0    5  
4  ...  3  2   1  2     0     0     0     0     1    5  

[5 rows x 63 columns]
数据维度：(34017, 63)


In [9]:
df = df.drop("所在楼层", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 62)


In [10]:
# 处理建筑面积
def extract_area_value(text):
    """    
    参数:
    text: str - 建筑面积字符串，如"$ 282.02m^{2} $"
    
    返回:
    float - 提取出的面积数值
    """
    match = re.search(r"(\d+\.?\d*)", text)
    
    if match:
        return float(match.group(1))
    else:
        return 0  # 如果没有匹配到数字，返回0

In [11]:
df['建筑面积值'] = df['建筑面积'].apply(extract_area_value)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

Index(['ID', '城市', '区域', '板块', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例',
       '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息',
       '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县',
       '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值'],
      dtype='object')
        ID  城市     区域      板块     建筑面积     套内面积 房屋朝向  建筑结构 装修情况  梯户比例  ... 厅数  \
0  1000000   0  109.0   367.0  282.02㎡      NaN  南 北  钢混结构   简装  两梯五户  ...  2   
1  1000001   0   28.0   606.0   88.42㎡   71.78㎡  南 北  钢混结构   其他  一梯两户  ...  1   
2  1000002   0  123.0  1110.0  175.52㎡  139.86㎡   西北  混合结构   简装  两梯六户  ...  1   
3  1000003   0   65.0   555.0  106.13㎡      NaN  南 北  钢混结构   其他  一梯两户  ...  1   
4  1000004   0  109

In [12]:
df = df.drop("建筑面积", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 62)


In [13]:
# 检查缺失值
print(df['套内面积'].isnull().sum())
# 填充缺失值为空字符串
df['套内面积'] = df['套内面积'].fillna('')
df['套内面积值'] = df['套内面积'].apply(extract_area_value)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

24102
Index(['ID', '城市', '区域', '板块', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯',
       '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势',
       '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm',
       '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值'],
      dtype='object')
        ID  城市     区域      板块     套内面积 房屋朝向  建筑结构 装修情况  梯户比例 配备电梯  ... 厨房数 卫数  \
0  1000000   0  109.0   367.0           南 北  钢混结构   简装  两梯五户    有  ...   1  2   
1  1000001   0   28.0   606.0   71.78㎡  南 北  钢混结构   其他  一梯两户    有  ...   1  1   
2  1000002   0  123.0  1110.0  139.86㎡   西北  混合结构   简装  两梯六户    有  ...   1  2   
3  1000003   0   65.0   555.0           南 北  钢混结构   其他  一梯两户    无  ...   1  1   
4  1000004   0  109

In [14]:
df = df.drop("套内面积", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 62)


In [15]:
# 处理房屋朝向
def extract_room_direction(text):
    south = north = west = east = 0
    
    if "南" in text:
        south = 1
    if "北" in text:
        north = 1
    if "西" in text:
        west = 1
    if "东" in text:
        east = 1
    
    return south, north, west, east

In [16]:
# 检查缺失值
print(df['房屋朝向'].isnull().sum())
# 填充缺失值为空字符串
df['房屋朝向'] = df['房屋朝向'].fillna('')
# 应用提取函数并创建新列
df[['向南', '向北', '向西', '向东']] = df['房屋朝向'].apply(
    lambda x: pd.Series(extract_room_direction(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型',
       '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点',
       '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置',
       '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值',
       '向南', '向北', '向西', '向东'],
      dtype='object')
        ID  城市     区域      板块 房屋朝向  建筑结构 装修情况  梯户比例 配备电梯 别墅类型  ... 是否为中层  \
0  1000000   0  109.0   367.0  南 北  钢混结构   简装  两梯五户    有  NaN  ...     1   
1  1000001   0   28.0   606.0  南 北  钢混结构   其他  一梯两户    有  NaN  ...     1   
2  1000002   0  123.0  1110.0   西北  混合结构   简装  两梯六户    有  NaN  ...     0   
3  1000003   0   65.0   555.0  南 北  钢混结构   其他  一梯两户    无  NaN  ...     1   
4  1000004   0  109.0   9

In [17]:
df = df.drop("房屋朝向", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 65)


In [18]:
# 处理建筑结构
def extract_room_structure(text):
    hunhe = zhuanhun = kuangjia = zhuanmu = ganghun = gang = 0
    if "混合" in text:
        hunhe = 1
    if "砖混" in text:
        zhuanhun = 1
    if "框架" in text:
        kuangja = 1
    if "砖木" in text:
        zhuanmu = 1
    if "钢混" in text:
        ganghun = 1
    elif "钢" in text:
        gang = 1
    
    return hunhe, zhuanhun, kuangjia, zhuanmu, ganghun, gang

In [19]:
# 检查缺失值
print(df['建筑结构'].isnull().sum())
# 填充缺失值为空字符串
df['建筑结构'] = df['建筑结构'].fillna('')
# 应用提取函数并创建新列
df[['混合', '砖混', '框架', '砖木', '钢混', '钢']] = df['建筑结构'].apply(
    lambda x: pd.Series(extract_room_structure(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

14
Index(['ID', '城市', '区域', '板块', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间',
       '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍',
       '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别',
       '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费',
       '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位',
       '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北',
       '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢'],
      dtype='object')
        ID  城市     区域      板块  建筑结构 装修情况  梯户比例 配备电梯 别墅类型        交易时间  ... 向南  \
0  1000000   0  109.0   367.0  钢混结构   简装  两梯五户    有  NaN  2025-02-21  ...  1   
1  1000001   0   28.0   606.0  钢混结构   其他  一梯两户    有  NaN  2025-01-17  ...  1   
2  1000002   0  123.0  1110.0  混合结构   简装  两梯六户    有  NaN  2025-04-03  ...  0   
3  1000003   0   65.0   555.0  钢混结构   其他  一梯两户    无  NaN 

In [20]:
df = df.drop("建筑结构", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 70)


In [21]:
# 处理装修情况
def extract_room_fitment(text):
    jingzhuang = jianzhuang = qita = maopi = 0
    if "精装" in text:
        jingzhuang = 1
    if "简装" in text:
        jianzhuang = 1
    if "其他" in text:
        qita = 1
    if "毛坯" in text:
        maopi = 1

    
    return jingzhuang, jianzhuang, qita, maopi

In [22]:
# 检查缺失值
target_column = "装修情况"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['精装装修', '简装装修', '其他装修', '毛坯装修']] = df[target_column].apply(
    lambda x: pd.Series(extract_room_fitment(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

14
Index(['ID', '城市', '区域', '板块', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属',
       '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代',
       '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修',
       '毛坯装修'],
      dtype='object')
        ID  城市     区域      板块 装修情况  梯户比例 配备电梯 别墅类型        交易时间  交易权属  ... 混合  \
0  1000000   0  109.0   367.0   简装  两梯五户    有  NaN  2025-02-21    私产  ...  0   
1  1000001   0   28.0   606.0   其他  一梯两户    有  NaN  2025-01-17   商品房  ...  0   
2  1000002   0  123.0  1110.0   简装  两梯六户    有  NaN  2025-04-03   商品房  ...  1   
3  1000003   0   65.0   55

In [23]:
df = df.drop("装修情况", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 73)


In [24]:
def convert_chinese_number(chinese_num):
    """
    将中文数字转换为阿拉伯数字
    
    参数:
    chinese_num: str - 中文数字字符串
    
    返回:
    int - 转换后的阿拉伯数字
    """
    chinese_to_arabic = {
        '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
        '六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
        '两': 2
    }
    
    # 处理十以上的数字
    if '十' in chinese_num:
        parts = chinese_num.split('十')
        if len(parts) == 1:  # 十
            return 10
        elif len(parts) == 2:
            if parts[0] == '' and parts[1] == '':  # 这种情况不应该发生
                return 10
            elif parts[0] == '':  # 十一
                return 10 + chinese_to_arabic.get(parts[1], 0)
            elif parts[1] == '':  # 二十
                return chinese_to_arabic.get(parts[0], 0) * 10
            else:  # 二十五
                return chinese_to_arabic.get(parts[0], 0) * 10 + chinese_to_arabic.get(parts[1], 0)
    
    # 单个数字
    return chinese_to_arabic.get(chinese_num, 0)

In [25]:
def extract_ratio(text):
    """
    从梯户比例字符串中提取电梯数和户数，并计算梯户比
    
    参数:
    text: str - 梯户比例字符串，如"一梯三户"、"两梯八户"
    
    返回:
    float - 梯户比（户数/电梯数）
    """
    # 使用正则表达式匹配模式：X梯Y户
    pattern = r'(\S+)梯(\S+)户'
    match = re.search(pattern, text)
    
    if match:
        elevator_chinese = match.group(1)  # 电梯数的中文表示
        household_chinese = match.group(2)  # 户数的中文表示
        
        # 转换为阿拉伯数字
        elevator_num = convert_chinese_number(elevator_chinese)
        household_num = convert_chinese_number(household_chinese)
        
        # 计算梯户比（户数/电梯数）
        if elevator_num > 0:
            return household_num / elevator_num
        else:
            return -1  # 避免除以零错误
    else:
        return -1  # 如果匹配失败，返回0

In [26]:
# 检查缺失值
target_column = "梯户比例"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数创建新列
df['梯户比'] = df[target_column].apply(extract_ratio)
print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

635
Index(['ID', '城市', '区域', '板块', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易',
       '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行',
       'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商',
       '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修',
       '梯户比'],
      dtype='object')
        ID  城市     区域      板块  梯户比例 配备电梯 别墅类型        交易时间  交易权属        上次交易  \
0  1000000   0  109.0   367.0  两梯五户    有  NaN  2025-02-21    私产  2008-02-02   
1  1000001   0   28.0   606.0  一梯两户    有  NaN  2025-01-17   商品房  2010-12-18   
2  1000002   0  123.0  1110.0  两梯六户    有  NaN  2025-04-03   商品房  2010-05-07   
3  1000003   0   65.0   555.0 

In [27]:
df = df.drop("梯户比例", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 73)


In [28]:
# 处理电梯情况
def extract_elevator(text):
    is_elev = -1
    if "有" in text:
        is_elev = 1
    if "无" in text:
        is_elev = 0
    
    return is_elev

In [29]:
# 检查缺失值
target_column = "配备电梯"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['是否有电梯']] = df[target_column].apply(
    lambda x: pd.Series(extract_elevator(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

4092
Index(['ID', '城市', '区域', '板块', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途',
       '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon',
       'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯'],
      dtype='object')
        ID  城市     区域      板块 配备电梯 别墅类型        交易时间  交易权属        上次交易  房屋用途  \
0  1000000   0  109.0   367.0    有  NaN  2025-02-21    私产  2008-02-02  普通住宅   
1  1000001   0   28.0   606.0    有  NaN  2025-01-17   商品房  2010-12-18  普通住宅   
2  1000002   0  123.0  1110.0    有  NaN  2025-04-03   商品房  2010-05-07  普通住宅   
3  1000003   0   65.0   555.

In [30]:
df = df.drop("配备电梯", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 73)


In [31]:
# 处理别墅类型
def extract_bighouse(text):
    is_expensive = is_cheap = 0
    if "独栋" in text or "双拼" in text:
        is_expensive = 1
    if "叠拼" in text or "联排" in text:
        is_cheap = 1
    
    return is_expensive, is_cheap

In [32]:
# 检查缺失值
target_column = "别墅类型"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['独栋或双拼', "联排或叠拼"]] = df[target_column].apply(
    lambda x: pd.Series(extract_bighouse(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

33863
Index(['ID', '城市', '区域', '板块', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限',
       '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat',
       '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数',
       '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水',
       '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈',
       '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层',
       '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木',
       '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼',
       '联排或叠拼'],
      dtype='object')
        ID  城市     区域      板块 别墅类型        交易时间  交易权属        上次交易  房屋用途 房屋年限  \
0  1000000   0  109.0   367.0       2025-02-21    私产  2008-02-02  普通住宅  满五年   
1  1000001   0   28.0   606.0       2025-01-17   商品房  2010-12-18  普通住宅  满五年   
2  1000002   0  123.0  1110.0       2025-04-03   商品房  2010-05-07  普通住宅  满五年   
3  1000003   0   

In [33]:
df = df.drop("别墅类型", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 74)


In [34]:
# 检查缺失值
target_column = "交易时间"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')

# 将"交易时间"列转换为datetime类型
df['交易时间dt'] = pd.to_datetime(df[target_column])
# 设置参考日期为2025年12月31日
reference_date = datetime(2025, 12, 31)
# 计算每个交易日期距离参考日期的天数
df['交易距今'] = (reference_date - df['交易时间dt']).dt.days

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属',
       '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份',
       '区县', '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混',
       '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼',
       '交易时间dt', '交易距今'],
      dtype='object')
        ID  城市     区域      板块        交易时间  交易权属        上次交易  房屋用途 房屋年限 产权所属  \
0  1000000   0  109.0   367.0  2025-02-21    私产  2008-02-02  普通住宅  满五年  非共有   
1  1000001   0   28.0   606.0  2025-01-17   商品房  2010-12-18  普通住宅  满五年  非共有   
2  1000002   0  123.0  1110.0  2025-04-03   商品房  2010-05-07  普通住宅  满五年  非共有   
3  1000003 

In [35]:
df = df.drop("交易时间", axis=1)
df = df.drop("交易时间dt", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 74)


In [36]:
# 先处理所有超出范围的日期
def fix_out_of_bounds_dates(date_str):
    """
    修复超出 pandas 时间范围的日期字符串
    """
    try:
        # 尝试转换为 datetime 来检查是否超出范围
        pd.to_datetime(date_str)
        return date_str
    except:
        # 提取日期部分
        parts = date_str.split('-')
        if len(parts) >= 3:
            # 将年份改为在范围内的年份（例如当前年份）
            fixed_year = '2020'  # 替换为合理的年份
            return f"{fixed_year}-{parts[1]}-{parts[2]}"
        return date_str

In [38]:
# 检查缺失值
target_column = "上次交易"
print(df[target_column].isnull().sum())
# 应用修复函数
df[target_column] = df[target_column].apply(fix_out_of_bounds_dates)
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')


# 将"交易时间"列转换为datetime类型
df['上次交易时间dt'] = pd.to_datetime(df[target_column])
# 设置参考日期为2025年12月31日
reference_date = datetime(2025, 12, 31)
# 计算每个交易日期距离参考日期的天数
df['上次交易距今'] = (reference_date - df['上次交易时间dt']).dt.days
df['上次交易距今'] = df['上次交易距今'].fillna(-1)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息',
       '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县',
       '板块_comm', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混',
       '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼',
       '交易距今', '上次交易时间dt', '上次交易距今'],
      dtype='object')
        ID  城市     区域      板块  交易权属        上次交易  房屋用途 房屋年限 产权所属  抵押信息  ...  \
0  1000000   0  109.0   367.0    私产  2008-02-02  普通住宅  满五年  非共有   NaN  ...   
1  1000001   0   28.0   606.0   商品房  2010-12-18  普通住宅  满五年  非共有   NaN  ...   
2  1000002   0  123.0  1110.0   商品房  2010-05-07  普通住宅  满五年  非共有   NaN  ...   
3  1000003 

In [39]:
count_negative = (df['上次交易距今'] < 0).sum()
print("小于0的值（异常数据）数量:", count_negative)

小于0的值（异常数据）数量: 7243


In [40]:
df = df.drop("上次交易", axis=1)
df = df.drop("上次交易时间dt", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 74)


In [41]:
# 处理房屋年限
def extract_room_year(text):
    low_two = two_five = high_five = 0
    if "满五年" in text:
        high_five = 1
    if "未满两年" in text:
         low_two = 1
    elif "满两年" in text:
        two_five = 1
    
    return low_two, two_five, high_five

In [42]:
# 检查缺失值
target_column = "房屋年限"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['房屋低于两年', "房屋二到五年", "房屋满五年"]] = df[target_column].apply(
    lambda x: pd.Series(extract_room_year(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

11296
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势',
       '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm',
       '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值',
       '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修',
       '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今',
       '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途 房屋年限 产权所属  抵押信息      房屋优势  ... 毛坯装修  \
0  1000000   0  109.0   367.0    私产  普通住宅  满五年  非共有   NaN    、房本满五年  ...    0   
1  1000001   0   28.0   606.0   商品房  普通住宅  满五年  非共有   NaN  地铁、房本满五年  ...    0   
2  1000002   0  123.0  1110.0   商品房  普通住宅  满五年  非共有   NaN  装修、房本满五年

In [43]:
df = df.drop("房屋年限", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 76)


In [44]:
# 处理产权所属
def extract_room_right(text):
    is_shared = -1
    if "非" in text:
        is_shared = 0
    elif "共有" in text:
         is_shared = 1
    
    return is_shared

In [45]:
# 检查缺失值
target_column = "产权所属"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[["是否共有产权"]] = df[target_column].apply(
    lambda x: pd.Series(extract_room_right(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '产权所属', '抵押信息', '房屋优势', '核心卖点',
       '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置',
       '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值',
       '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修',
       '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今',
       '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年', '是否共有产权'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途 产权所属  抵押信息      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅  非共有   NaN    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  非共有   NaN  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  非共有   NaN  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅  非共有  

In [46]:
df = df.drop("产权所属", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 76)


In [47]:
df = df.drop("抵押信息", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [48]:
def extract_building_year(building_year):
    """
    参数:
    building_year: str - 建筑年代字符串
    
    返回:
    float - 处理后的年份值
    """    
    # 转换为字符串类型
    building_year = str(building_year)
    
    # 2002-2006年
    if '-' in building_year:
        # 使用正则表达式提取两个年份
        years = re.findall(r'\d+', building_year)
        start_year = int(years[0])
        end_year = int(years[1])
        return (start_year + end_year) / 2
    
    # 2004年
    elif '年' in building_year:
        # 提取年份数字
        year_match = re.search(r'\d+', building_year)
        return float(year_match.group())

    return -1

In [49]:
# 检查缺失值
target_column = "建筑年代"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[["房屋年份"]] = df[target_column].apply(
    lambda x: pd.Series(extract_building_year(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9406
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '建筑年代',
       '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修',
       '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅 

In [50]:
df = df.drop("建筑年代", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [51]:
# 检查缺失值
target_column = "房屋总数"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1户')
# 使用正则表达式提取数字
df['总户数'] = df[target_column].str.extract(r'(\d+)').astype(int)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3715
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修',
       '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅  

In [52]:
df = df.drop("房屋总数", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [53]:
# 检查缺失值
target_column = "楼栋总数"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1栋')
# 使用正则表达式提取数字
df['总楼数'] = df[target_column].str.extract(r'(\d+)').astype(int)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3715
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅  地

In [54]:
df = df.drop("楼栋总数", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [55]:
# 检查缺失值
target_column = "绿 化 率"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('0.0')
# 提取数字
df['绿化率'] = df[target_column].str.rstrip('%').astype(float)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9180
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水',
       '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈',
       '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层',
       '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木',
       '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼',
       '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年', '是否共有产权',
       '房屋年份', '总户数', '总楼数', '绿化率'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅  地铁

In [56]:
df = df.drop("绿 化 率", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [57]:
# 检查缺失值
target_column = "容 积 率"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1')
# 提取数字
df['容积率'] = df[target_column].astype(float)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9303
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '容 积 率', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混',
       '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼',
       '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年', '是否共有产权', '房屋年份', '总户数',
       '总楼数', '绿化率', '容积率'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅  地铁、房

In [58]:
df = df.drop("容 积 率", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [59]:
def extract_property_fee(value):
    """
    参数:
    value: str - 物业费字符串
    
    返回:
    float - 提取的物业费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/月/㎡"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/月/㎡'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1.0
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1.0
    except (ValueError, AttributeError):
        return -1.0

In [60]:
# 检查缺失值
target_column = "物 业 费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["物业费"]] = df[target_column].apply(
    lambda x: pd.Series(extract_property_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

8489
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '物 业 费', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费',
       '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数',
       '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值',
       '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢',
       '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼',
       '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年', '是否共有产权', '房屋年份', '总户数',
       '总楼数', '绿化率', '容积率', '物业费'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅  地铁、房本满

In [61]:
df = df.drop("物 业 费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 75)


In [62]:
# 处理供水
def extract_water_giving(text):
    minshui = shangshui = 0
    if "民水" in text:
        minshui = 1
    if "商水" in text:
        shangshui = 1
    
    return minshui, shangshui

In [63]:
# 检查缺失值
target_column = "供水"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民水供水', '商水供水']] = df[target_column].apply(
    lambda x: pd.Series(extract_water_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

8195
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值',
       '向南', '向北', '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修',
       '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今',
       '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数',
       '绿化率', '容积率', '物业费', '民水供水', '商水供水'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   990.0  已购公房  普通住宅 

In [64]:
df = df.drop("供水", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 76)


In [65]:
# 处理供暖
def extract_warm_giving(text):
    jizhong = zi = wu = 0
    if "集中供暖" in text:
        jizhong = 1
    if "自采暖" in text:
        zi = 1
    if "无供暖" in text:
        wu = 1
    
    return jizhong, zi, wu

In [66]:
# 检查缺失值
target_column = "供暖"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['集中供暖', '自采暖', '无供暖']] = df[target_column].apply(
    lambda x: pd.Series(extract_warm_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

18602
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '供暖', '供电', '燃气费', '供热费', '停车位',
       '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北',
       '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修',
       '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年',
       '房屋二到五年', '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费',
       '民水供水', '商水供水', '集中供暖', '自采暖', '无供暖'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  109.0   9

In [67]:
df = df.drop("供暖", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 78)


In [68]:
# 处理供电
def extract_elec_giving(text):
    mindian = shangdian = 0
    if "民电" in text:
        mindian = 1
    if "商电" in text:
        shangdian = 1
    
    return mindian, shangdian

In [69]:
# 检查缺失值
target_column = "供电"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民电供电', '商电供电']] = df[target_column].apply(
    lambda x: pd.Series(extract_elec_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

8177
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '供电', '燃气费', '供热费', '停车位',
       '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北',
       '向西', '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修',
       '毛坯装修', '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年',
       '房屋二到五年', '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费',
       '民水供水', '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   0  

In [70]:
df = df.drop("供电", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [71]:
def extract_burn_fee(value):
    """
    参数:
    value: str - 燃气费字符串
    
    返回:
    float - 提取的燃气费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/m³"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/m³'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1
    except (ValueError, AttributeError):
        return -1

In [72]:
# 检查缺失值
target_column = "燃气费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["燃气费数值"]] = df[target_column].apply(
    lambda x: pd.Series(extract_burn_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9157
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修',
       '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004   

In [73]:
df = df.drop("燃气费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [74]:
def extract_warm_fee(value):
    """
    参数:
    value: str - 燃气费字符串
    
    返回:
    float - 提取的燃气费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/㎡"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/㎡'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1
    except (ValueError, AttributeError):
        return -1

In [75]:
# 检查缺失值
target_column = "供热费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["供热费数值"]] = df[target_column].apply(
    lambda x: pd.Series(extract_warm_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

23028
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  1000004

In [76]:
df = df.drop("供热费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [77]:
def extract_park(value):
    """
    参数:
    value: str - 停车位字符串
    
    返回:
    float - 提取的停车位数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == '':
        return -1.0
    else:
        return float(value)

In [78]:
# 检查缺失值
target_column = "停车位"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["停车位数量"]] = df[target_column].apply(
    lambda x: pd.Series(extract_park(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9571
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值',
       '停车位数量'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4 

In [79]:
df = df.drop("停车位", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [80]:
def extract_parking_fee(value):
    """
    参数:
    value: str - 停车费字符串
    
    返回:
    float - 提取的停车费数值，或-1.0（如果无法提取）
    """
    # 检查是否为空值
    if value == '':
        return -1.0
    
    # 转换为字符串并去除前后空格
    value_str = str(value).strip()
    
    # 检查是否只包含数字（整数或小数）
    # 使用正则表达式匹配纯数字（可包含小数点）
    pattern = r'^-?\d*\.?\d+$'
    if re.match(pattern, value_str):
        try:
            return float(value_str)
        except ValueError:
            return -1.0
    else:
        return -1.0

In [81]:
# 检查缺失值
target_column = "停车费用"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["停车费"]] = df[target_column].apply(
    lambda x: pd.Series(extract_parking_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

9922
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', '房屋优势', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', '停车费用', 'coord_x', 'coord_y',
       '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层', '是否为高层',
       '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合', '砖混',
       '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比', '是否有电梯',
       '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年', '房屋满五年',
       '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水', '商水供水',
       '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值', '停车位数量', '停车费'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途      房屋优势  \
0  1000000   0  109.0   367.0    私产  普通住宅    、房本满五年   
1  1000001   0   28.0   606.0   商品房  普通住宅  地铁、房本满五年   
2  1000002   0  123.0  1110.0   商品房  普通住宅  装修、房本满五年   
3  1000003   0   65.0   555.0   商品房  普通住宅    、房本满五年   
4  100000

In [82]:
df = df.drop("停车费用", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [83]:
def comment_len(text):
    # 检查是否为空值
    return len(text)

In [84]:
# 检查缺失值
target_column = "房屋优势"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["房屋优势分数"]] = df[target_column].apply(
    lambda x: pd.Series(comment_len(x), dtype='object')
)

6379


In [85]:
# 检查缺失值
target_column = "核心卖点"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["核心卖点分数"]] = df[target_column].apply(
    lambda x: pd.Series(comment_len(x), dtype='object')
)

8048


In [86]:
# 检查缺失值
target_column = "户型介绍"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["户型介绍分数"]] = df[target_column].apply(
    lambda x: pd.Series(comment_len(x), dtype='object')
)

24005


In [87]:
# 检查缺失值
target_column = "周边配套"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["周边配套分数"]] = df[target_column].apply(
    lambda x: pd.Series(comment_len(x), dtype='object')
)

15031


In [88]:
# 检查缺失值
target_column = "交通出行"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["交通出行分数"]] = df[target_column].apply(
    lambda x: pd.Series(comment_len(x), dtype='object')
)

15358


In [89]:
df = df.drop("房屋优势", axis=1)
df = df.drop("核心卖点", axis=1)
df = df.drop("户型介绍", axis=1)
df = df.drop("周边配套", axis=1)
df = df.drop("交通出行", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 79)


In [90]:
def get_rights(text):
    high_price = middle_price = low_price = 0
    high_lst = ["商品房", "央产房", "私产"]
    middle_lst = ["已购公房", "房改房", "集资房", "限价商品房", "拆迁还建房", "经济适用房", "售后公房"]
    low_lst = ["动迁安置房", "一类经济适用房", "二类经济适用房", "定向安置房", "自住型商品房"]
    if text in high_lst:
        high_price = 1
    if text in middle_lst:
        middle_price = 1
    if text in low_lst:
        low_price = 1
    return high_price, middle_price, low_price

In [91]:
# 检查缺失值
target_column = "交易权属"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['高价交易类型', '中价交易类型', '低价交易类型']] = df[target_column].apply(
    lambda x: pd.Series(get_rights(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '交易权属', '房屋用途', 'lon', 'lat', '年份', '区县',
       '板块_comm', '环线位置', '物业类别', '开发商', '物业公司', '建筑结构_comm', '物业办公电话', '产权描述',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西',
       '向东', '混合', '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修',
       '梯户比', '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值', '停车位数量',
       '停车费', '房屋优势分数', '核心卖点分数', '户型介绍分数', '周边配套分数', '交通出行分数', '高价交易类型',
       '中价交易类型', '低价交易类型'],
      dtype='object')
        ID  城市     区域      板块  交易权属  房屋用途         lon        lat      年份  \
0  1000000   0  109.0   367.0    私产  普通住宅  117.389491  40.901030  2022.0   
1  1000001   0   28.0   606.0   商品房  普通住宅  117.376625  40.767478  2022.0   
2  1000002   0  123.

In [92]:
df = df.drop("交易权属", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 81)


In [93]:
def get_uses(text):
    high_price = middle_price = low_price = 0
    high_lst = ["别墅", "花园洋房", "新式里弄", "老公寓"]
    middle_lst = ["普通住宅", "公寓", "公寓（住宅）", "公寓/住宅", "公寓/公寓", "商务公寓", "商务型公寓", "酒店式公寓"]
    low_lst = ["商住两用", "商业办公类", "商业", "车库"]
    if text in high_lst:
        high_price = 1
    if text in middle_lst:
        middle_price = 1
    if text in low_lst:
        low_price = 1
    return high_price, middle_price, low_price

In [94]:
# 检查缺失值
target_column = "房屋用途"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['高价房屋用途', '中价房屋用途', '低价房屋用途']] = df[target_column].apply(
    lambda x: pd.Series(get_uses(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '区域', '板块', '房屋用途', 'lon', 'lat', '年份', '区县', '板块_comm',
       '环线位置', '物业类别', '开发商', '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值', '停车位数量',
       '停车费', '房屋优势分数', '核心卖点分数', '户型介绍分数', '周边配套分数', '交通出行分数', '高价交易类型',
       '中价交易类型', '低价交易类型', '高价房屋用途', '中价房屋用途', '低价房屋用途'],
      dtype='object')
        ID  城市     区域      板块  房屋用途         lon        lat      年份     区县  \
0  1000000   0  109.0   367.0  普通住宅  117.389491  40.901030  2022.0  109.0   
1  1000001   0   28.0   606.0  普通住宅  117.376625  40.767478  2022.0   28.

In [95]:
df = df.drop("房屋用途", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 83)


In [96]:
# 处理建筑结构_comm
def extract_structure_comm(text):
    talou = banlou = pingfang = 0
    if "塔楼" in text:
        talou = 1
    if "板楼" in text:
        banlou = 1
    if "平房" in text:
        pingfang = 1
    if "塔板结合" in text:
        talou = 1
        banlou = 1
        
    return talou, banlou, pingfang

In [97]:
# 检查缺失值
target_column = "建筑结构_comm"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['塔楼', '板楼', '平房']] = df[target_column].apply(
    lambda x: pd.Series(extract_structure_comm(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

7834
Index(['ID', '城市', '区域', '板块', 'lon', 'lat', '年份', '区县', '板块_comm', '环线位置',
       '物业类别', '开发商', '物业公司', '建筑结构_comm', '物业办公电话', '产权描述', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '套内面积值', '向南', '向北', '向西', '向东', '混合',
       '砖混', '框架', '砖木', '钢混', '钢', '精装装修', '简装装修', '其他装修', '毛坯装修', '梯户比',
       '是否有电梯', '独栋或双拼', '联排或叠拼', '交易距今', '上次交易距今', '房屋低于两年', '房屋二到五年',
       '房屋满五年', '是否共有产权', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '民水供水',
       '商水供水', '集中供暖', '自采暖', '无供暖', '民电供电', '商电供电', '燃气费数值', '供热费数值', '停车位数量',
       '停车费', '房屋优势分数', '核心卖点分数', '户型介绍分数', '周边配套分数', '交通出行分数', '高价交易类型',
       '中价交易类型', '低价交易类型', '高价房屋用途', '中价房屋用途', '低价房屋用途', '塔楼', '板楼', '平房'],
      dtype='object')
        ID  城市     区域      板块         lon        lat      年份     区县  板块_comm  \
0  1000000   0  109.0   367.0  117.389491  40.901030  2022.0  109.0    367.0   
1  1000001   0   28.0   606.0  117.376625  40.767478 

In [98]:
df = df.drop("建筑结构_comm", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(34017, 85)


In [99]:
df = df.drop("物业类别", axis=1)
df = df.drop("产权描述", axis=1)

In [100]:
df = df.drop("开发商", axis=1)
df = df.drop("物业公司", axis=1)
df = df.drop("物业办公电话", axis=1)

In [101]:
df = df.drop("环线位置", axis=1)

In [102]:
df.to_csv(f"结构化数据_{data_batch}_{data_target}.csv", index=False)